In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

torch.set_printoptions(linewidth=200)

In [2]:
hidden_size = 200   # 根据实际情况决定
num_moves = 311   # =动作数量=文件数量

class JointsDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.moves = []
        for i in range(num_moves):
            with open('./train_data/' + str(i) + '.txt') as f:
                content = f.read()
                frames = content.splitlines()
                self.moves.append(frames)
    
    def __getitem__(self, index):
        move = self.moves[index]
        y = torch.empty(1, dtype=torch.long)  #label
        y[0] = int(move[0])
        frams_list = []
        for frame in move[1:]:
            joints_str = frame.split('\t')
            
            if len(joints_str)>96:
                joints_str = joints_str[:-3]
            joints_flt = [float(joint) for joint in joints_str]
            frams_list.append(joints_flt)
        x = torch.Tensor(frams_list)
        return x,y
    
    def __len__(self):
        return len(self.moves)

In [3]:
trn_ds = JointsDataset()
print(trn_ds[0])
print(trn_ds[0][0].shape)
print(len(trn_ds))

(tensor([[-987.9500,  657.8600, 2552.3999,  ...,    8.0000,   56.0000,    0.0000],
        [ 737.0000,  677.6300, 2634.1001,  ...,  424.0000,   64.0000,    0.0000],
        [-991.9200,  660.8000, 2554.0000,  ...,    8.0000,   58.0000,    0.0000],
        ...,
        [-142.4800,  598.8900, 2918.8999,  ...,  248.0000,   92.0000,    0.0000],
        [-384.9500,  658.7200, 2332.3999,  ...,  104.0000,   36.0000,    0.0000],
        [-178.6100,  597.3600, 2936.3000,  ...,  252.0000,   92.0000,    0.0000]]), tensor([0]))
torch.Size([92, 96])
311


In [4]:

class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()
        # TO DO
        # YOUR CODE HERE
        
        self.in_size = 96
        self.hidden_size = 200
        self.out_size = 3
        self.i2h = nn.Linear(self.in_size, self.hidden_size)
        self.h2h = nn.Linear(self.hidden_size,self.hidden_size)
        
       
    
    def forward(self,a,x):
        # TO DO
        # YOUR CODE HERE
        x = x.view(self.in_size)
        waa_a = self.h2h(a)
        wax_x = self.i2h(x)
        new_a = torch.tanh(waa_a + wax_x)
        # y = self.h2o(new_a)
        return new_a

In [5]:
in_size = 96
hidden_size = 200
out_size = 3
def train_one_epoch(model, loss_fn, optimizer, h_prev):
    # Go through the training examples one at a time
    count = 0
    num_correct = 0
    for move_num, (x,y) in enumerate(trn_dl):
        
        count += 1
        print('----',count,'----')
        
        y = y.view(y.shape[1])
        model.train()
        loss = 0
        optimizer.zero_grad()
        for i in range(x.shape[1]):
            a = model.forward(h_prev,x[0][i])
            print(x[0][i])
            h_prev = a 
        h2o = nn.Linear(hidden_size,out_size)
        out = h2o(h_prev)
        y_pred = F.softmax(out)
        label = y.view(1)
        print('y_pred', y_pred)
        print('label',label)
        l = loss_fn(y_pred, label)
        loss += l   
        correct = get_correct(y_pred,label)
        if correct == True:
            num_correct += 1
        print('num of correct',num_correct)
        # Backpropagate through time
        # YOUR CODE HERE
        loss.backward(retain_graph=True)
        
        # Clip your gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        
        # Update parameters
        # YOUR CODE HERE
        optimizer.step()
        print('loss',loss)
        
        
    accuracy = num_correct/count
    print('ACCURACY',accuracy)
    print('LOSS',loss)

In [6]:
# utility for getting prediction accuracy
def get_correct(y_pred, y):
    # find the prediction class label
    _ ,pred_class = y_pred.max(dim=1)
    return pred_class == y

In [7]:
trn_ds = JointsDataset()
trn_dl = DataLoader(trn_ds, batch_size=1, shuffle=True)

def train(trn_ds, trn_dl, epochs=1):
    # Create a new model, loss_fn and optimizer.
    # YOUR CODE HERE
    model = RNN()
    # Use cross entropy loss
    loss_fn = nn.CrossEntropyLoss()
    # Use Adam
    optimizer = optim.Adam(model.parameters(),lr=0.01)
    h_prev = torch.zeros((1,hidden_size))
    for e in range(1, epochs+1):
        print(f'{"-"*20} Epoch {e} {"-"*20}')
        print('h_prev', h_prev)
        
       
        count = 0
        num_correct = 0
        for move_num, (x,y) in enumerate(trn_dl):
        
            count += 1
            print('----',count,'----')
        
            y = y.view(y.shape[1])
            model.train()
            loss = 0
            optimizer.zero_grad()
            for i in range(x.shape[1]):
                a = model.forward(h_prev,x[0][i])
                h_prev = a 
            print('h_prev',h_prev)
            h2o = nn.Linear(hidden_size,out_size)
            out = h2o(h_prev)
            y_pred = F.softmax(out)
            label = y.view(1)
            print('y_pred', y_pred)
            print('label',label)
            l = loss_fn(y_pred, label)
            loss += l   
            correct = get_correct(y_pred,label)
            if correct == True:
                num_correct += 1
            print('num of correct',num_correct)
            # Backpropagate through time
            # YOUR CODE HERE
            loss.backward(retain_graph=True)
        
            # Clip your gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        
            # Update parameters
            # YOUR CODE HERE
            optimizer.step()
            print('loss',loss)
        
        
        accuracy = num_correct/count
        print('ACCURACY',accuracy)
        print('LOSS',loss)

In [ ]:
#Start training
train(trn_ds, trn_dl, epochs=10)

-------------------- Epoch 1 --------------------
h_prev tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])
---- 1 ----
h_prev tensor([[ 1.0000, -1.0000,  1.0000,  1.0000, -1.0000, -1.0000, -1.0000,  1.0000,  1.00

C:\Users\jx880\Anaconda3\envs\pytorch37\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


 tensor(0.7409, grad_fn=<AddBackward0>)
---- 3 ----
h_prev tensor([[ 1.0000, -1.0000,  1.0000,  1.0000, -1.0000, -1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000,
          1.0000, -1.0000, -1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000,
          1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000,
         -1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000,
          1.0000,  0.9997,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.00

loss tensor(0.9718, grad_fn=<AddBackward0>)
---- 8 ----
h_prev tensor([[ 1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,
         -1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1.,
          1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,
         -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1.,
          1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1.,  1.,  1

loss tensor(1.3153, grad_fn=<AddBackward0>)
---- 13 ----
h_prev tensor([[ 1.0000, -1.0000,  1.0000,  1.0000, -1.0000, -1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000,
          1.0000, -1.0000, -1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000,
          1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  1.0000, -1.0000, -1.0000,  1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000,
         -1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000,
          1.0000, -1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000,  1.0000, -1.0000, 

...

loss tensor(1.2594, grad_fn=<AddBackward0>)
---- 45 ----
h_prev tensor([[-1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,
         -1.,  1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1.,
          1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,
         -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1.,
          1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,  1., -1.,  1., -

loss tensor(1.1606, grad_fn=<AddBackward0>)
---- 52 ----
h_prev tensor([[-1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,
         -1.,  1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1.,
          1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,
         -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1.,
          1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,  1., -1.,  1., -

loss tensor(1.0182, grad_fn=<AddBackward0>)
---- 59 ----
h_prev tensor([[-1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,
         -1.,  1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1.,
          1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,
         -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1.,
          1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,  1., -1.,  1., -

loss tensor(1.2945, grad_fn=<AddBackward0>)
---- 66 ----
h_prev tensor([[-1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,
         -1.,  1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1.,
          1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,
         -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1.,
          1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,  1., -1.,  1., -

loss tensor(1.1001, grad_fn=<AddBackward0>)
---- 73 ----
h_prev tensor([[-1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,
         -1.,  1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1.,
          1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,
         -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1.,
          1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,  1., -1.,  1., -